# eBay Marketplace Fetching Script

This script is designed to fetch product details from the eBay marketplace. However, **this fetching system is not actively used** because running large-scale data collection, storage, and recommendation for millions of users would require significant server resources. 

Instead, this script serves as a reference for fetching product data and can be used for small-scale testing or integration into a larger system with appropriate infrastructure.

THIS CODE CELL GIVES YOU AN OAUTH TOKEN FROM YOUR EBAY CREDENTIALS FOR EXTRACTING DATA

In [ ]:
import requests
import base64

CLIENT_ID = "client_id"
CLIENT_SECRET = "client_secret"

credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

url = "https://api.ebay.com/identity/v1/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {encoded_credentials}"
}
body = {
    "grant_type": "client_credentials",
    "scope": "https://api.ebay.com/oauth/api_scope"
}

response = requests.post(url, headers=headers, data=body)

if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("✅ Your eBay Access Token:", access_token)
else:
    print("❌ Error:", response.text)

#The code shows an error because the credentials are not entered 
#Put in the credentials to display the OAuth token

❌ Error: {"error":"invalid_client","error_description":"client authentication failed"}


THIS CODE CELL FETCHES THE PRODUCTS FROM THE US EBAY MARKET

In [2]:
import requests

EBAY_ACCESS_TOKEN = access_token

EBAY_BASE_URL = "https://api.ebay.com/buy/browse/v1/item_summary/search"

def fetch_ebay_products(keyword, max_results=10):
    headers = {
        "Authorization": f"Bearer {EBAY_ACCESS_TOKEN}",
        "X-EBAY-C-MARKETPLACE-ID": "EBAY_US",
        "Content-Type": "application/json"
    }

    params = {
        "q": keyword,
        "limit": max_results
    }

    response = requests.get(EBAY_BASE_URL, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return f"❌ Error: {response.status_code}, Response: {response.text}"

products = fetch_ebay_products("laptop", max_results=5)
print(products)


{'href': 'https://api.ebay.com/buy/browse/v1/item_summary/search?q=laptop&limit=5&offset=0', 'total': 1010544, 'next': 'https://api.ebay.com/buy/browse/v1/item_summary/search?q=laptop&limit=5&offset=5', 'limit': 5, 'offset': 0, 'itemSummaries': [{'itemId': 'v1|387635594054|0', 'title': 'SGIN 15.6 Inch Laptop 4GB RAM 128GB SSD Computer with 2.4G/5G WiFi  Up to 2.5GHz', 'leafCategoryIds': ['177'], 'categories': [{'categoryId': '177', 'categoryName': 'PC Laptops & Netbooks'}, {'categoryId': '58058', 'categoryName': 'Computers/Tablets & Networking'}, {'categoryId': '175672', 'categoryName': 'Laptops & Netbooks'}], 'image': {'imageUrl': 'https://i.ebayimg.com/images/g/Of8AAOSw8VFmQxBr/s-l225.jpg'}, 'price': {'value': '139.99', 'currency': 'USD'}, 'itemHref': 'https://api.ebay.com/buy/browse/v1/item/v1%7C387635594054%7C0', 'seller': {'username': 'sgin-shop-1121', 'feedbackPercentage': '99.2', 'feedbackScore': 127}, 'marketingPrice': {'originalPrice': {'value': '559.96', 'currency': 'USD'}, '

ITERATES THROUGH ALL THE FETCHED PRODUCTS AND DISPLAYS THEM IN A USER READABLE MANNER

In [3]:
def display_products(products):
    """Formats and prints eBay product search results in a user-friendly format."""
    if isinstance(products, str):
        print(products)
        return

    items = products.get("itemSummaries", []) 

    if not items:
        print("❌ No products found!")
        return

    print("\n🔍 eBay Search Results:")
    print("=" * 50)

    for i, item in enumerate(items, start=1):
        title = item.get("title", "N/A")
        price_info = item.get("price", {})
        price = price_info.get("value", "N/A")
        currency = price_info.get("currency", "N/A")
        url = item.get("itemWebUrl", "#")

        print(f"{i}. 🏷️ {title}")
        print(f"   💰 Price: {price} {currency}")
        print(f"   🔗 Link: {url}\n")


display_products(products)



🔍 eBay Search Results:
1. 🏷️ SGIN 15.6 Inch Laptop 4GB RAM 128GB SSD Computer with 2.4G/5G WiFi  Up to 2.5GHz
   💰 Price: 139.99 USD
   🔗 Link: https://www.ebay.com/itm/387635594054?_skw=laptop&hash=item5a40e1c346:g:Of8AAOSw8VFmQxBr&amdata=enc%3AAQAKAAAA4PeG5RIuIyokJHJy903%2F5UbKlY0iRRMQqzMWIE1rUO7qrUHB4H25YnwSH2SvAhht0HCscz8zDx7v79QQoePhvJ2ROnJ4kfxgn8fREqEOxoM6BKh9%2B27xMCsfJD3JHRAL%2Bqa0HTlsx6WKVzoDqk8iLlDQBezFARzex%2BweA1Z6B7Feftav83nJA9Ns6Lq78kYOsV94nWAHCnCiDID1k3pPugLNKR8cUg50Mx%2F%2B11dcX%2FDGkxdQNF8ZOdECU3nXBT1C%2FdDSZUu0SzHxBAG6kalC4xF5ZF9QxPhVS4jvawLU%2B9yb

2. 🏷️ HP ProBook 640 G5 Laptop 14" Windows 11 Quad-Core i5 4.1GHz 8GB RAM 128GB SSD
   💰 Price: 172.38 USD
   🔗 Link: https://www.ebay.com/itm/254582474636?_skw=laptop&hash=item3b464c478c:g:EhEAAOSwM6hn0vKp&amdata=enc%3AAQAKAAAA4PeG5RIuIyokJHJy903%2F5UZVsSkdSqaRiFKaDLJTDzhD6Lwza0a8WGYQSdPvWlKDJzkGI7M3f9Cn3r5TUSKH7iJOjaZcOYs5dKUg7pH3TdLFy2rQN%2FU5LXx1cMEbqe4oaAMLFk3QHRqyFCowdX8n1C1ARGmD%2FqTjCY901XfZXjaegJsIlIRjzKOVwG5lQ%2